## The Report of Study Case
written by Selcuk Emre Ozer


___

This report is written to show deficiencies of study case.

___

Deficiencies are going to be reviewed in 2 titles.

### Titles:
* SQL Deficincies
* Altair Deficincies

In [35]:
# Libraries
import pandas as pd
import altair as alt

In [42]:
# connect to bigquery
import json
from google.oauth2 import service_account
from google.cloud import bigquery

path = '/Users/selcu/Desktop/bruin/bruin-hiring-1b27c94e981a.json'

with open(path, 'r') as f:
    json_key_file = json.load(f)
    
credentials = service_account.Credentials.from_service_account_info(json_key_file)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

___

### Question 2

In the study case, I got help from AI tools(ChatGPT). In this situation I could not learn basic of SQL properly. You can check the SQL deficincies which I have improved below this part.

#### SQL Part

In [ ]:
# QUESTION 2: Who are the top contributors?
# mistake is to prefer name to count top contrubitors?

SELECT 
    committer.name, <-------MISTAKE
    COUNT(*) as counter

FROM 
    table
GROUP BY 
    committer.name
ORDER BY 
    counter desc
LIMIT 10

People might have the same name, so to find top contributors properly, even though they have the same name, we should prefer a unique feature for each contrubitors. It is mail address. Each people has their own unique email address.

##### Correct SQL Code

In [417]:
query = """
SELECT 
    committer.name,
    committer.email,  -- CORRECT PREFERENCE
    COUNT(*) as mail_counter
FROM
     `bruin-hiring.github_repos.sample_commits`
GROUP BY
    committer.name,
    committer.email
ORDER BY
    mail_counter desc
LIMIT
    10
"""
query_job = client.query(query)
results = query_job.result()

In [419]:
x2 = [] # for chart, creating x and y axis lists
y2 = []


print("RESULTS:\n_______\n")

for row in results:
    x2.append(row[0]+' '+row[1][40:])
    y2.append(row[2])
    print(row[2],row[0],'->' , row[1],'\n')


RESULTS:
_______

49523 David S. Miller -> fe08d3c717adf2ae63592e4c9aec6e3e404d8e3e@davemloft.net 

40480 Linus Torvalds -> 69652caca27c8b940640ad396ab71f93cacec34f@linux-foundation.org 

37969 Greg Kroah-Hartman -> 4645f7897fd33786a2ee1264d590b3c400559d85@linuxfoundation.org 

19605 Greg Kroah-Hartman -> 4645f7897fd33786a2ee1264d590b3c400559d85@suse.de 

18875 John W. Linville -> 2a53bac7a5d324865ef46ec4c38b2c0fba1456b4@tuxdriver.com 

13372 Mauro Carvalho Chehab -> ad86ba2154032c9f55743a190faa2459a9d61d42@redhat.com 

13311 Ingo Molnar -> 9dbbbf0688fedc85ad4da37637f1a64b8c718ee2@elte.hu 

12197 Linus Torvalds -> 69652caca27c8b940640ad396ab71f93cacec34f@g5.osdl.org 

8928 Linus Torvalds -> 69652caca27c8b940640ad396ab71f93cacec34f@woody.linux-foundation.org 

8661 Takashi Iwai -> 4596b3305151c7ee743192a95d394341e3d3b644@suse.de 



You can see there is more than one Linus Torvalds

___

In [423]:
#### Altair Part(with mistakes)
df2 = pd.DataFrame({'contributors':x2,'number of contribution':y2})

alt.Chart(df2).mark_bar().encode(x = 'contributors', y = 'number of contribution',
                                color = alt.condition(
                                alt.datum.contributors == 'Mark Brown',
                                alt.value('orange'),
                                alt.value('steelblue')
                                )).properties(width=400)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

#### Altair Part

You can see in new code, top cont. list does not include 'Mark Brown' as name. Because this Altair part is static and this is not convinent for changeable data cases. To fix this code >

In [427]:
#### Altair Fixed Part
max_index = df2['number of contribution'].idxmax() # max index

alt.Chart(df2).mark_bar().encode(x = 'contributors', y = 'number of contribution',
                                color = alt.condition(
                                alt.datum.contributors == df2['contributors'][max_index],
                                alt.value('orange'),
                                alt.value('steelblue')
                                )).properties(width=400)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In addition to sort these data.<br>
(Detail is alternative using :<br>
    encode(x = 'contributors'... -> useless structure<br>
    encode(x = alt.X('contributors')... -> to add sorting
)

In [430]:
#### In addition
max_index = df2['number of contribution'].idxmax() # max index

alt.Chart(df2).mark_bar().encode(x = alt.X('contributors').sort('-y'),
                                 y =alt.Y('number of contribution'),
                                color = alt.condition(
                                alt.datum.contributors == df2['contributors'][max_index],
                                alt.value('orange'),
                                alt.value('steelblue')
                                )).properties(width=400)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

---

### Question 3

3- Are there any developer contributing to more than 1 repos?


#### SQL Part

In [ ]:
"""
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q3 AS
    WITH developer_contribute as (
    SELECT
      committer.name as name,
      COUNT(DISTINCT repo_name) as repo_count
    FROM `github_repos.sample_commits`
    GROUP BY
      name
    )
    SELECT
      developer_contribute.name as name,
      developer_contribute.repo_count as repo_count
    FROM developer_contribute
    WHERE
      repo_count > 1;
"""

This part works correctly but it can be shorter

In [ ]:
"""
CREATE OR REPLACE TABLE bruin-hiring.cdt_selcukemreozer.q3 as
SELECT
    committer.name as name,
    COUNT(DISTINCT repo_name) as repo_count
FROM 
    `github_repos.sample_commits`
GROUP BY 
    name
HAVING -- using of having provides a shorter query
    repo_count > 1
ORDER BY 
    repo_count DESC
"""

The result is the same but query is shorter.

#### Altair part

In [382]:
query = """
SELECT 
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q3;
"""
query_job = client.query(query)
results = query_job.result()


x3 = [] # for chart, creating x and y axis lists
y3 = []

for row in results:
    x3.append(row[0])
    y3.append(row[1])
print("data is ready!")

data is ready!


In [396]:
df = pd.DataFrame({'name':x3,'number of diffirent repos':y3})
alt.Chart(df).mark_bar().encode(x='name',y='number of diffirent repos')

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Above code there is an unaesthetic view, to fix that(sort):

In [390]:
alt.Chart(df).mark_bar().encode(x=alt.X('name').sort('-y'),y=alt.Y('number of diffirent repos'))

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

---

Question 4 has the same problem with Question 3

---

### Question 5

Number of commits per day

In [441]:
query = """
SELECT 
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q5
"""

query_job = client.query(query)
results = query_job.result()

x5 = [] # for chart, creating x and y axis lists
y5 = []


for row in results:
    x5.append(row[0])
    y5.append(row[1])
print('query is ready')

query is ready


In [496]:
df5 = pd.DataFrame({'date':x5,'number of commits':y5})
df5["date"] = pd.to_datetime(df5["date"])

Above code line, the point is that date should not be a string type, to is scale by altair library. It must be date type. (.to_datetime())

In [472]:
# an example using
data = ['01-01-2023']
df_date = pd.DataFrame({'date_string': data})

df_date['date'] = pd.to_datetime(df_date['date_string'], format='%d-%m-%Y')


In [470]:
df_date

,date_string,date
0,01-01-2023,2023-01-01


In [541]:
alt.Chart(df5, width = 1200).mark_point().encode(x='date',y='number of commits')

alt.Chart(...)

The alternative chart types are line and bar

---

### Question 6

Number of commits per weekday

---

#### SQL Part

FORMAT_DATE using:

In [ ]:
committer.date
"""   
    Row	       date

     1          2013-06-11 11:50:00 UTC
     2          2014-07-21 16:11:40 UTC
     3          2015-06-26 04:34:20 UTC
     4          2015-06-26 04:39:59 UTC
     5          2006-03-22 04:31:14 UTC
"""
    FORMAT_DATE('%A', committer.date)

 -> give us weekdays as day names

In [ ]:
"""
    Row	       date

     1          Friday
     2          Wednesday
     3          Sunday
    ...
"""

---

In [ ]:
FORMAT_DATE('%d', committer.date) 

-> give us days as numbers

In [ ]:
"""
    Row	       date

     1          01
     2          11
     3          31
    ...
"""                    

---

In [ ]:
FORMAT_DATE('%B',committer.date) -> give us months as month names
FORMAT_DATE('%m',committer.date) -> give us months as numbers

FORMAT_DATE('%C') -> century??
FORMAT_DATE('%y') -> give us years as numbers 

ex.
'02' means '2002'

to visualize it as full year: 

FORMAT_DATE('20%y') -> output is '2002'

----

#### Altair Part

In [505]:
query = """
SELECT
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q6
"""

query_job = client.query(query)
results = query_job.result()

x6 = list()
y6 = list()

print("RESULTS:\n_______\n")

for row in results:
    x6.append(row[0])
    y6.append(row[1])
    print(row[0],'->',row[1])

RESULTS:
_______

Sunday -> 43072
Saturday -> 44814
Friday -> 112297
Thursday -> 114669
Monday -> 115061
Wednesday -> 120722
Tuesday -> 121674


In [512]:

df6 = pd.DataFrame({'days':x6,'count':y6})
df6


,days,count
0,Sunday,43072
1,Saturday,44814
2,Friday,112297
3,Thursday,114669
4,Monday,115061
5,Wednesday,120722
6,Tuesday,121674


In this dataset days are not ordered as weekdays. We should fix it in pandas.

In [524]:
df6.index = df6['days']
df6 = df6.loc[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]

To order days as weekdays
df6 = df6.loc[['Monday','Tuesday',...

In [572]:
alt.Chart(df6).mark_bar().encode(x=alt.X('days'),y=alt.Y('count'))

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

To prevent, default alphabetical sort in Altair lib, You should use alt.X(x, sort = None)

In [532]:
alt.Chart(df6).mark_bar().encode(x=alt.X('days',sort=None),y=alt.Y('count'))

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)